In [33]:
import bs4
from datetime import datetime

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.keys import Keys

In [18]:
def openBrowser(business, headless=False):
    global driver
    try:
        options = webdriver.ChromeOptions()
        options.add_argument('profile')
        options.add_argument('user-data-dir=./profiles/' + business)
        
        if headless == True:
            options.add_argument('headless')
            options.add_argument('--window-size-1920x1080')
            
        mainUrl = 'https://web.whatsapp.com/'
        driver = webdriver.Chrome(options=options, executable_path='./webdriver/win/chromedriver.exe')
        driver.get(mainUrl)
        return True
    except Exception as error:
        print('Error ocurred in OpenBrowser: {} {}'.format(type(error), error))
        return False

In [112]:
# FUNCOES PEQUENAS PARA NAO PRECISAR REPETIR CODIGO
def getNow():
    '''
        Funcao para retornar tempo e data atual formatado
    '''
    return datetime.now().strftime('%d/%m/%y %H:%M:%S')

def getPageSource():
    global htmlPage
    htmlPage = bs4.BeautifulSoup(driver.page_source, features='html.parser')
    return True, 'Atualiazdo'

In [44]:
# VERIFICA SE O WHATSAPP ESTA CONECTADO 
def verifyConnection():
    '''
        Verifica se a div alerta existe e qual o seu conteudo, se for 'Phone not connected' retorna False,
        caso contrario retorna True para conectado.
    '''
    
    htmlPage = bs4.BeautifulSoup(driver.page_source, features='html.parser')
    try:
        span_alert = htmlPage.find('span',{'class':'_3O0po'})
        text_alert = span_alert.find('div',{'class':'_28Bny'}).text
        if text_alert == 'Phone not connected':
            return False
        else:
            return True
    except Exception as error:
        do = verifyError('verifyConnection', error, getNow())

In [113]:
def getChats():
    '''
        Pega as conversas que estao carregadas no documento atual (posicao do cursos na pagina)
    '''
    getPageSource() #atualiza o pagesource em HtmlPage
    chats = htmlPage.find_all('div',{'class':'X7YrQ'})
    return chats

In [141]:
# FUNCOES QUE IRAO PEGAR INFORMACOES DE CADA CHAT
def getChatName(chat):
    '''
        Recebe a div completa de cada chat da lista e ira retornar o "titulo"/ nome da conversa 
        (titulo para caso nao esteja salvo o contato)
    '''
    try:
        contact_name = chat.find('div', {'class':'_3H4MS'}).text
        assert contact_name != '', 'Algum erro ou Sem Nome de Contato'
        return contact_name
    except Exception as error:
        do = verifyError('getChatName', error, getNow())
        
def getLabel(chat):
    '''
        Recebe um chat e verifica se ha algum label nele, se ha ira retornar o qual label foi encontrado
    
        # TODO: Verificar no banco o que cada label significa para a empresa em questao
    '''
    try:
        label = chat.find('div',{'class':'yKiIK'})
        label = label.find('path').get('fill')
        return label
    except Exception as error:
        do = verifyError('getLabel', error, getNow())
        pass

def isGroup(chat):
    '''
        Rececebe um chat e verifica se eh um grupo
    
        Quado eh grupo nao tera uma div especifica, entao retornara False
    '''
    try:
        div_contact = chat.find('span',{'class':'_3NWy8'})
        return True if div_contact == None else False
    except Exception as error:
        do = verifyError('isGroup', error, getNow())
        pass

In [246]:
# FUNCOES PRA TRATAR O TEMPO E STATUS DAS MENSAGENS, VIA CHAT OU HISTORICO
def getTime(content, chat_or_message):
    '''
        Recebe um chat ou uma mensagem e ira devolver o tempo da mensagem,
        se for um chat ira retornar o horario da ultima mensagem enviada ou recebida,
        se for mensagem ira retornar o time da mensagem
    '''
    # TODO: inserir tratamento de horario para dias da semana e para formatos de horarios diferentes
    try:
        if chat_or_message == 'chat':
            time = content.find('div',{'class':'_0LqQ'}).text
            return time
        elif chat_or_message == 'message':
            return 'need implementation'
        else:
            return 'content need to be chat or message'
    except Exception as error:
        do = verifyError('getTime', error, getNow())
        pass

def getMsgStatus(content, chat_or_message):
    '''
        Recebe uma mensagem de chat ou do historico e retorna seu status,
        se for mensagem enviada ira retornar se foi somente enviada, entregue ou recebida,
        se for recebida ira retornar "received"
    '''
    try:
        if chat_or_message == 'chat':
            status_label = chat.find('div',{'class':'_3VIru'})
            return 'Received' if status_label == None else 'Sent'
        elif chat_or_message == 'message':
            return 'need implementation'
    except Exception as error:
        do = verifyError('getMsgStatus', error, getNow())
        pass

In [244]:
chats = getChats()

In [245]:
for pos, chat in enumerate(chats):
    print(pos, getChatName(chat), getMsgStatus(chat, 'chat'), getTime(chat, 'chat'))

0 Data Science LIVRE Received 9:59 PM
1 Save Save Save Sent yesterday
2 Mae Received yesterday
3 Gabriel Zatorski Received yesterday
4 Daniel Saulo Bitcoins Sent 4:51 AM
5 PyNerds  Received 7:38 PM
6 +55 47 9616-6553 Received 1:25 PM
7 Xaropes Received 10:08 PM
8 33 Sushi House Received yesterday
9 Data Science Talk Received 3:42 PM
10 Cueca Rodrigo Santana Received 1:41 PM
11 Programação em Python Sent 9:40 PM
12 Andrew Brasil Advocacia Balneario Received 10:00 PM
13 Carmen Mae Gui Sent 6:46 PM
14 Guilherme Kussler Received 7:36 PM
15 Nycolas Machado Sent 1:53 PM
16 Helen Sent 6:19 PM


In [143]:
# FUNCAO DE VERIFICACAO DE ERROS
def verifyError(where, error, when):
    '''
        Verificacao de erros e excecoes 
    '''
    pass
#     print('Error ocurred in {} at {}: {} {}'.format(where, when, type(error), error))

In [ ]:
openBrowser('nomadbitcoin')